# Mini GPT Zero — Construção Didática de um Modelo de Linguagem

Este notebook tem como objetivo entender profundamente, passo a passo, como funciona um modelo de linguagem do tipo **GPT** (*Generative Pretrained Transformer*), sem depender de bibliotecas complexas.

Foco didático: **cada parte do código será explicada em detalhes**, para solidificar tanto o entendimento teórico quanto a prática de implementação.  
*Este notebook está em construção contínua como parte de um estudo autodirigido para reduzir débito técnico em IA, LLMs e fundamentos de deep learning.*

---

## Organização do notebook

1. **Título e introdução do projeto**
2. **Seção de bibliotecas**
3. **Upload ou leitura do corpus**
4. **Tokenização e preparação dos dados**
5. **Função de batching**
6. **Construção do modelo passo a passo (camadas do Transformer)**
7. **Treinamento básico**
8. **Geração de texto**
9. **Conclusão e próximos passos**

---

## Com explicações comentadas

- Cada célula de código será seguida de um texto explicando o que foi feito, por que foi feito e como funciona.

---

## Objetivo final

- Criar um **notebook funcional e didático**, que servirá como base para um artigo técnico documentando todo o processo.


In [ ]:
# Mini GPT Zero — Construindo um GPT didático no Google Colab
## Documentação interativa e código comentado do projeto.

# Nova seção

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Config:
    vocab_size = 113         # tamanho do vocabulário (ajuste com base no seu)
    block_size = 8           # tamanho da sequência de entrada
    n_embed = 64             # dimensão do embedding
    n_heads = 4              # número de cabeças de atenção
    n_layers = 2             # número de blocos transformer
    dropout = 0.1            # taxa de dropout

config = Config()

# --------------------------
# Embedding + Atenção
# --------------------------

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(config.n_embed, head_size, bias=False)
        self.query = nn.Linear(config.n_embed, head_size, bias=False)
        self.value = nn.Linear(config.n_embed, head_size, bias=False)
        self.dropout = nn.Dropout(config.dropout)
        self.register_buffer("tril", torch.tril(torch.ones(config.block_size, config.block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)     # (B, T, C)
        q = self.query(x)   # (B, T, C)
        wei = q @ k.transpose(-2, -1) / (C ** 0.5)  # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

# --------------------------
# Multi-Head Attention
# --------------------------

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(config.n_embed, config.n_embed)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# --------------------------
# Feedforward
# --------------------------

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(config.dropout),
        )

    def forward(self, x):
        return self.net(x)

# --------------------------
# Bloco Transformer
# --------------------------

class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# --------------------------
# Modelo GPT simplificado
# --------------------------

class MiniGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(config.vocab_size, config.n_embed)
        self.position_embedding_table = nn.Embedding(config.block_size, config.n_embed)
        self.blocks = nn.Sequential(*[Block(config.n_embed, config.n_heads) for _ in range(config.n_layers)])
        self.ln_f = nn.LayerNorm(config.n_embed)
        self.lm_head = nn.Linear(config.n_embed, config.vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)             # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))  # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            return logits
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss

In [ ]:
# Monta o drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

# Declaração da classe MiniGPT
model = MiniGPT().to(device)

# Caminho do corpus no drive
# DATA_PATH = Path("/content/drive/caminho-para-seus-arquivos-no-drive-nem-fundendo-que-exponho-o-meu-aqui")

# Hiperparâmetros iniciais
block_size = 8   # tamanho do contexto (número de caracteres anteriores)
batch_size = 4   # número de sequências por batch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Baixar o arquivo corpus.txt do GitHub usando wget
!wget -O corpus.txt "https://raw.githubusercontent.com/ivano-lab/mini-gpt-zero/refs/heads/main/data/corpus.txt"

# 1. Leitura do texto
with open('corpus.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 2. Tokenização caractere a caractere
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocabulário: {vocab_size} caracteres")

# Mapas de conversão
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

# Funções auxiliares
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Texto codificado
data = torch.tensor(encode(text), dtype=torch.long)

print("Trecho original:")
print(text[:2000])
print("\nCodificado:")
print(encode(text[:2000]))

# 3. Criação dos batches
def get_batch(split):
    # separa entre treino e validação (90/10)
    split_idx = int(0.9 * len(data))
    data_split = data[:split_idx] if split == 'train' else data[split_idx:]

    ix = torch.randint(len(data_split) - block_size, (batch_size,))
    x = torch.stack([data_split[i:i+block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

# Exemplo de uso
x_batch, y_batch = get_batch('train')
print("Entrada (x):")
print(x_batch)
print("Como texto:")
for i in range(batch_size):
    print(decode(x_batch[i].tolist()), "->", decode(y_batch[i].tolist()))

model = MiniGPT().to(device)  # importante enviar o modelo para o dispositivo correto (CPU ou GPU)

# Rodar o modelo com um forward pass usando o mesmo batch já obtido
logits, loss = model(x_batch, y_batch)

print("Forma do input (x):", x_batch.shape)
print("Forma do target (y):", y_batch.shape)
print("Forma dos logits:", logits.shape)
print("Loss inicial:", loss.item())

--2025-06-26 01:29:59--  https://raw.githubusercontent.com/ivano-lab/mini-gpt-zero/refs/heads/main/data/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 418217 (408K) [text/plain]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>] 408.42K  --.-KB/s    in 0.04s   

2025-06-26 01:29:59 (9.01 MB/s) - ‘corpus.txt’ saved [418217/418217]

Vocabulário: 113 caracteres
Trecho original:
﻿The Project Gutenberg eBook of Dom Casmurro
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If 

# Nova seção

## Definição do modelo Transformer (MiniGPT)
Aqui implementamos as classes e funções que compõem o modelo de linguagem. Este modelo é inspirado no GPT e é responsável por:
- Criar os embeddings dos tokens
- Aplicar atenção
- Processar as saídas do Transformer
- Gerar logits e loss para o treinamento


# Nova seção